## 进入 MMagic 主目录

In [1]:
import os
os.chdir('mmagic')

## 导入工具包

In [2]:
import cv2
import numpy as np
import mmcv
from mmengine import Config
from PIL import Image

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules

register_all_modules()

## 毛胚房图片转Canny 边缘检测图——为了测试生成边缘的效果

In [30]:
img = cv2.imread("data/maopei/maopei3.jpg")
 # 高斯模糊  降低噪声
blurred = cv2.GaussianBlur(img, (5, 5), 0)
# 转为灰度图像
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
# 计算x y 方向梯度
grad_x = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)
grad_y = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)

 
cv2.imwrite("data/maopei/maopei3_canny.jpeg", cv2.Canny(grad_x,grad_y, 60,120))
 

True

## 载入ControlNet模型

In [10]:
cfg = Config.fromfile('configs/controlnet/controlnet-canny.py')
controlnet = MODELS.build(cfg.model).cuda()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


06/17 22:24:27 - mmengine - INFO - Set model dtype to 'torch.float32'.


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


06/17 22:24:55 - mmengine - INFO - Creating runwayml/stable-diffusion-v1-5 by 'HuggingFace'


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Do not support Xformers. Please install Xformers first. The program will run without Xformers.


06/17 22:24:59 - mmengine - INFO - Set model dtype to 'torch.float32'.
Do not support Xformers. Please install Xformers first. The program will run without Xformers.


## 输入Canny边缘图

In [46]:
control_url = 'data/maopei/maopei3.jpg'
control_img = mmcv.imread(control_url)
# 高斯模糊  降低噪声
blurred = cv2.GaussianBlur(control_img, (5, 5), 0)
# 转为灰度图像
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
# 计算x y 方向梯度
grad_x = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)
grad_y = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)
control = cv2.Canny(grad_x,grad_y,60,120)
control = control[:, :, None]
control = np.concatenate([control] * 3, axis=2)
control = Image.fromarray(control)

## 咒语Prompt

In [49]:
prompt = 'Room with nordic style.'

## 执行预测

In [52]:
output_dict = controlnet.infer(prompt, control=control)
samples = output_dict['samples']
for idx, sample in enumerate(samples):
    sample.save(f'output/sample_{idx}.png')
controls = output_dict['controls']
for idx, control in enumerate(controls):
    control.save(f'output/control_{idx}.png')

100%|██████████| 20/20 [00:13<00:00,  1.51it/s]
